# Load libraries and Themes

In [1]:
suppressPackageStartupMessages({
    suppressWarnings({library(Seurat)
        library(SeuratObject)
        library(SeuratDisk)
        library(ggplot2)
        library(tidyverse)
        library(harmony)
        library(DESeq2)
        library(EnhancedVolcano)
        library(Rsamtools)
        library(svglite)
        library(viridis)
        library(gridExtra)
        library(scCustomize)
        library(ggpubr)
        library(pals)
        library(patchwork)
    })})

In [ ]:
#Color Palettes

palette.doublets <- c(
    "#197278",#turqu
    "#C1C1C1"#grey
)

palette.2 <- c(
    "#990902",
    "#0303a3"
)

palette.4 <- DiscretePalette(4, palette = "stepped", shuffle = TRUE)

palette.12 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette.15 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette <- c(
    "#990902",#CM_0
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#660202",#PER
    "#4b048a",#MESO
    "#7d0701",#CM_1
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#d90f04",#CM_2
    "#aad902"#BC
)
   
palette.treatment <- c(
    "#AF0000",#ALDO
    "#C1C1C1"#CTRL
)

In [ ]:
umap_theme <- theme(
  axis.line=element_blank(),
  axis.text.x=element_blank(),
  axis.text.y=element_blank(),
  axis.ticks=element_blank(),
  axis.title.x=element_blank(),
  axis.title.y=element_blank(),
  panel.background=element_blank(),
  panel.border=element_blank(),
  panel.grid.major=element_blank(),
  panel.grid.minor=element_blank()
)

vln_theme_1 <- theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
  panel.background = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  text = element_text(size = 25)
)

vln_theme_2 <- theme(
  panel.background = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
)

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LA_CTRL_ALDO")

In [ ]:
set.seed(1234)

# Subcluster analysis

In [ ]:
obj_seu <- LoadH5Seurat("seurat_objects/obj_seu_merge_harmony_sgl_addmodule_downsample_rename.h5seurat")

In [ ]:
# Load after subcluster analysis is finished 

obj.subcluster_CM <- LoadH5Seurat("subcluster/Subcluster_CM_down.h5seurat")

## CM subcluster

In [ ]:
obj_harmony_addmodule <- LoadH5Seurat("seurat_objects/obj_seu_merge_harmony_sgl_addmodule.h5seurat")

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4, repr.plot.res = 300)

DimPlot(obj_harmony_addmodule, group.by = "seurat_clusters", label = TRUE) |
FeaturePlot(obj_harmony_addmodule, features = c("Zbtb16"))

In [ ]:
# Subset to CM clusters
CM <- subset(x = obj_harmony_addmodule, subset = seurat_clusters %in% c("0", "10", "16"))

In [ ]:
# Get genes with non-zero counts

counts <- GetAssayData(CM, layer = "counts")[,]
nonzero <- as.data.frame(rowSums(counts) > 0)
names(nonzero)[names(nonzero) == "rowSums(counts) > 0"] <- "nonzerofeature"
nonzero <- filter(nonzero, nonzerofeature == TRUE)
nonzero$names <- rownames(nonzero)
nonzero$nonzerofeature <- NULL
write.table(nonzero, "nonzerocounts/nonzerocounts_CM.csv", sep = ",", quote = FALSE,  row.names = FALSE, col.names = FALSE)

In [ ]:
tab <- table(CM@meta.data$sample_id)
print(tab)

In [ ]:
# Downsample to even cell numbers
downsampled_list <- list()

# Get unique sample IDs
sample_ids <- unique(CM@meta.data$sample_id)

# Loop through each sample ID
for (sample_id in sample_ids) {
  # Subset the object for the current sample_id
  subset_obj <- subset(CM, subset = sample_id == !!sample_id)

  # Downsample to a maximum of 7789 cells
  if (ncol(subset_obj) > 2137) {
    # Randomly select 7789 cells
    cell_indices <- sample(colnames(subset_obj), 2137)
    subset_obj <- subset(subset_obj, cells = cell_indices)
  }
  
  # Store the downsampled object in the list
  downsampled_list[[sample_id]] <- subset_obj
}

# Combine all downsampled objects back into one Seurat object
CM_down <- merge(downsampled_list[[1]], y = downsampled_list[-1])

# Check the counts of cells per sample_id
tab <- table(CM_down@meta.data$sample_id)
print(tab) 

In [ ]:
obj <- FindVariableFeatures(object = CM_down, selection.method = "vst")
obj <- ScaleData(object = obj, verbose = FALSE)
obj <- NormalizeData(object = obj, verbose = FALSE)
obj <- RunPCA(object = obj, verbose = FALSE)

In [ ]:
obj.subcluster_CM <- obj %>% 
  RunHarmony(group.by.vars = c("sex"), max_iter = 7, early_stop = FALSE, plot_convergence = TRUE, assay.use = "RNA", verbose = FALSE)

obj.subcluster_CM <- obj.subcluster_CM %>%
  RunUMAP(reduction = "harmony", dims = 1:35, verbose = FALSE, min.dist = 0.1, spread = 1) %>%
  FindNeighbors(reduction = "harmony", dims = 1:35, verbose = FALSE) %>%
  FindClusters(resolution = 0.35)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4, repr.plot.res = 300)

DimPlot(obj.subcluster_CM, group.by = "seurat_clusters") |
DimPlot(obj.subcluster_CM, group.by = "treatment")

In [ ]:
# Percentage of ALDO CMn in CM_0 and CM_1

obj.subcluster_CM@meta.data

In [ ]:
# Subset CM_0 and calculate percentage of ALDO CMn in CM_0
subset_CM_0 <- subset(obj.subcluster_CM, cell_type_sub == "CM_0")
aldo_CM_0 <- subset(subset_CM_0, treatment == "ALDO")
percent_aldo_CM_0 <- (ncol(aldo_CM_0) / ncol(subset_CM_0)) * 100

# Subset CM_1 and calculate percentage of ALDO CMn in CM_1
subset_CM_1 <- subset(obj.subcluster_CM, cell_type_sub == "CM_1")
aldo_CM_1 <- subset(subset_CM_1, treatment == "ALDO")
percent_aldo_CM_1 <- (ncol(aldo_CM_1) / ncol(subset_CM_1)) * 100

# Print the results
cat("Percentage of ALDO CMn in CM_0:", round(percent_aldo_CM_0, 2), "%\n")
cat("Percentage of ALDO CMn in CM_1:", round(percent_aldo_CM_1, 2), "%\n")

In [ ]:
all.markers <- FindAllMarkers(obj.subcluster_CM, only.pos = TRUE, min.pct = 0.50, logfc.threshold = 0.25, test.use = "wilcox")
all.markers %>%
  group_by("seurat_clusters") %>%
  slice_max(n = 10, order_by = avg_log2FC)

write.csv(all.markers, file = "subcluster/DEGs/all.markers_CM.csv")

In [ ]:
obj.subcluster_CM$cell_type <- Idents(obj.subcluster_CM)

In [ ]:
cluster_annotations_CM <- c("CM_0", "CM_1", "CM_2", "CM_3")
names(cluster_annotations_CM) <- levels(obj.subcluster_CM)
obj.subcluster_CM <- RenameIdents(obj.subcluster_CM, cluster_annotations_CM)
obj.subcluster_CM$cell_type_sub <- Idents(obj.subcluster_CM)

SaveH5Seurat(obj.subcluster_CM, "subcluster/Subcluster_CM_down")

In [ ]:
# without subsampling

obj <- FindVariableFeatures(object = CM, selection.method = "vst")
obj <- ScaleData(object = obj, verbose = FALSE)
obj <- NormalizeData(object = obj, verbose = FALSE)
obj <- RunPCA(object = obj, verbose = FALSE)

In [ ]:
obj.subcluster_CM_full <- obj %>% 
  RunHarmony(group.by.vars = c("sex"), max_iter = 7, early_stop = FALSE, plot_convergence = TRUE, assay.use = "RNA", verbose = FALSE)

obj.subcluster_CM_full <- obj.subcluster_CM_full %>%
  RunUMAP(reduction = "harmony", dims = 1:35, verbose = FALSE, min.dist = 0.1, spread = 1) %>%
  FindNeighbors(reduction = "harmony", dims = 1:35, verbose = FALSE) %>%
  FindClusters(resolution = 0.2)

options(repr.plot.width = 8, repr.plot.height = 4, repr.plot.res = 300)

DimPlot(obj.subcluster_CM_full, group.by = "seurat_clusters") |
DimPlot(obj.subcluster_CM_full, group.by = "treatment")

In [ ]:
tab <- table(obj.subcluster_CM_full@meta.data$sample_id)
print(tab)

In [ ]:
cluster_annotations_CM <- c("CM_0", "CM_1", "CM_2", "CM_3")
names(cluster_annotations_CM) <- levels(obj.subcluster_CM_full)
obj.subcluster_CM_full <- RenameIdents(obj.subcluster_CM_full, cluster_annotations_CM)
obj.subcluster_CM_full$cell_type_sub <- Idents(obj.subcluster_CM_full)

SaveH5Seurat(obj.subcluster_CM_full, "subcluster/Subcluster_CM_full")

In [ ]:
DimPlot(obj.subcluster_CM_full, group.by = "cell_type_sub") 

In [ ]:
# Visualize downsampled obj.subcluster_CM

options(repr.plot.width = 5, repr.plot.height = 2, repr.plot.res = 300) 

sex <- DimPlot(obj.subcluster_CM, pt.size = 0.5, group.by = "sex", shuffle = TRUE, cols = palette.2) + umap_theme & NoLegend()
sample_id <- DimPlot(obj.subcluster_CM, pt.size = 0.5, group.by = "sample_id", shuffle = TRUE, cols = palette.4) + umap_theme & NoLegend()
treatment <- DimPlot(obj.subcluster_CM, pt.size = 0.5, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) + umap_theme & NoLegend()
cell_type <- DimPlot(obj.subcluster_CM, group.by = "cell_type", pt.size = 0.5, label = TRUE, shuffle = TRUE, cols = palette.4, label.size = 10) + umap_theme & NoLegend()

ggsave("subcluster/Plots/CM_sex.svg", plot = sex, units = "cm", dpi = 300, width = 30, height = 20)
ggsave("subcluster/Plots/CM_sample_id.svg", plot = sample_id, units = "cm", dpi = 300, width = 30, height = 20)
ggsave("subcluster/Plots/CM_treatment.svg", plot = treatment, units = "cm", dpi = 300, width = 30, height = 20)
ggsave("subcluster/Plots/CM_cell_type.svg", plot = cell_type, units = "cm", dpi = 300, width = 30, height = 20)

cell_type|treatment
sample_id|sex

wrap_CM <- wrap_plots(cell_type, treatment, ncol = 2)
ggsave("subcluster/Plots/wrap_CM.svg", plot = wrap_CM, units = "cm", dpi = 300, width = 30, height = 15)

## DEG for downsampled

In [ ]:
markers_CM_1vs0 <- FindMarkers(obj.subcluster_CM, ident.1 = "CM_1", ident.2 = "CM_0", assay = "RNA", test.use = "wilcox", logfc.threshold = 0.25)

write.csv(markers_CM_1vs0, file = "subcluster/DEGs/markers_CM_1vs0.csv")

In [ ]:
# Subset the `sub_CM` object to include only cells with detectable Nr3c2 expression
obj.subcluster_CM_Nr3c2 <- subset(x = obj.subcluster_CM, subset = Nr3c2 > 2)

In [ ]:
markers_CM_1vs0_onlyNr3c2pos <- FindMarkers(obj.subcluster_CM_Nr3c2, ident.1 = "CM_1", ident.2 = "CM_0", assay = "RNA", test.use = "wilcox", logfc.threshold = 0.25)

write.csv(markers_CM_1vs0_onlyNr3c2pos, file = "subcluster/DEGs/markers_CM_1vs0_onlyNr3c2pos.csv")

In [ ]:
ls()

## Hcn4 positive cells

In [ ]:
# Define Hcn4-positive cells
hcn4_positive <- obj.subcluster_CM@assays$RNA@data["Hcn4", ] > 0

In [ ]:
# Get cluster assignments
cluster_info <- obj.subcluster_CM$cell_type_sub 

# Count total and Hcn4-positive cells per cluster
table_total <- table(cluster_info)
table_hcn4 <- table(cluster_info[hcn4_positive])

# Extract counts for CM_0 and CM_1
cm0_total <- table_total["CM_0"]
cm1_total <- table_total["CM_1"]
cm0_hcn4 <- table_hcn4["CM_0"]
cm1_hcn4 <- table_hcn4["CM_1"]

In [ ]:
# Fraction of Hcn4-positive cells in each cluster
frac_cm0_hcn4 <- cm0_hcn4 / cm0_total
frac_cm1_hcn4 <- cm1_hcn4 / cm1_total

# Ratio of Hcn4-positive cells in CM_0 vs. CM_1
hcn4_ratio_cm0_cm1 <- frac_cm0_hcn4 / frac_cm1_hcn4

# Absolute ratio of Hcn4-positive cells in CM_0 vs. CM_1
hcn4_ratio_cm0_cm1_abs <- cm0_hcn4/cm1_hcn4
# Print results
cat("Fraction of Hcn4+ cells in CM_0:", frac_cm0_hcn4, "\n")
cat("Fraction of Hcn4+ cells in CM_1:", frac_cm1_hcn4, "\n")
cat("Hcn4+ ratio CM_0 vs CM_1:", hcn4_ratio_cm0_cm1, "\n")
cat("Total Hcn4+ CM_0:", cm0_hcn4, "\n")
cat("Total Hcn4+ CM_1:", cm1_hcn4, "\n")
cat("Absolute Hcn4+ ratio CM_0 vs CM_1:", hcn4_ratio_cm0_cm1_abs, "\n")

In [2]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 24.04.1 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/scrna_dm/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] patchwork_1.3.0             pals_1.9                   
 [3] ggpubr_0.6.0                scCustomize_3.0.0          
 [5] gridExtra_2.3               viridis_0.6.5              
 [7] viridisLite_0.4.2           svglite_2.1.3              
 [9] Rsamtools_2.18.0